In [1]:
# !pip install tensorflow==2.0.0-alpha0

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn import model_selection, metrics
from sklearn.preprocessing import Normalizer
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras
import glob


import os
print(os.listdir("."))

['tf2_model2-Boosted.ipynb', '.ipynb_checkpoints', 'test.csv', 'train.csv', 'submission3.csv', 'Keras_-_PUBG.ipynb', 'sample_submission.csv', 'tf2_model1.ipynb', 'estimator_submission.csv', 'submission1.csv', 'estimator_api.ipynb', 'PUBG_submit.ipynb', 'Esti_works.ipynb', 'tf2_model3-Keras.ipynb', 'EDA.ipynb', 'Untitled.ipynb']


Using TensorFlow backend.


In [3]:
def load_data(train_path='train.csv', test_path='test.csv'):
    train = pd.DataFrame.from_csv(train_path, index_col="Id")
    test = pd.DataFrame.from_csv(test_path, index_col="Id")
    
    return train, test

In [4]:
train, test = load_data()

/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/home/lazukav/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
def preprocess(data):
    data.dropna(axis=0, inplace=True)
    if 'winPlacePerc' in data:
        data.winPlacePerc = np.where(data.winPlacePerc >= 0.5 , 1, 0)    
    return data

In [6]:
train = preprocess(train)
test = preprocess(test)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(train.drop('winPlacePerc', axis=1), train.winPlacePerc, test_size=0.2, shuffle=None)

In [8]:
# cols = [tf.feature_column.numeric_column(i) for i in X_train.select_dtypes(exclude=['object'])]
# for i in train.select_dtypes(include=['object']).columns.values:
#     cols.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(key=i, vocabulary_list=train[i].unique())))
# cols.append(matchType)
cols = [tf.feature_column.numeric_column(i) for i in ['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills']]

In [9]:
def train_classification_model(
    learning_rate,
    steps,
    batch_size,
    X_train,
    y_train,
    X_test,
    y_test,
    cols,
    m_dir,
    periods):

    steps_per_period = steps / periods  

    training_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_train,
        y = y_train,
        shuffle = True,
    )

    ptraining_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_train,
        y = y_train,
        num_epochs = 1,
        shuffle = False,
    )
    
    validation_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
        x = X_test,
        y = y_test,
        num_epochs = 1,
        shuffle = False,
    )

    classifier = tf.estimator.BoostedTreesClassifier(
        feature_columns=cols,
        n_batches_per_layer=batch_size,
        n_classes=2,
        n_trees=20,
        model_dir=m_dir
    )

    print("Training model...")
#     print("LogLoss error (on validation data):")
    training_errors = []
    validation_errors = []
    for period in range (0, periods):

        classifier.train(
            input_fn=training_input_fn,
            steps=steps_per_period
        )
        classifier.evaluate(input_fn=validation_input_fn)
        print("period:", period + 1)
    print("Done")

    return classifier

In [10]:
model = train_classification_model(
    learning_rate=0.01,
    steps=100,
    batch_size=20,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    cols=cols,
    m_dir='/tmp/model2',
    periods=5)

W0329 07:35:38.471166 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/training_util.py:238: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0329 07:35:38.503226 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0329 07:35:38.505985 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_functions.py:500: add_

Training model...
LogLoss error (on validation data):


W0329 07:35:38.804695 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:35:39.066659 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/monitored_session.py:877: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0329 07:35:39.250919 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:35:39.349242 139674315347712 meta_graph.py:447] Issue 

period: 1


W0329 07:36:26.979589 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:1069: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
W0329 07:36:27.264679 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:36:27.366013 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:36:27.806025 139674315347712 meta_graph.py:447] Issue encountered when s

...

period: 3


W0329 07:38:04.160445 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:38:04.252361 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:38:04.636664 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:38:05.683497 139674315347712 metrics_impl.py:783] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_i

period: 4


W0329 07:38:50.427281 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:38:50.509525 139674315347712 meta_graph.py:447] Issue encountered when serializing resources.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'_Resource' object has no attribute 'name'
W0329 07:38:50.854546 139674315347712 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/saver.py:965: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
W0329 07:38:50.884431 139674315347712 meta_graph.py:447] Issue encounter

period: 5
Done


In [11]:
scores3 = model.predict(input_fn=tf.compat.v1.estimator.inputs.pandas_input_fn(
    x = test,
    num_epochs = 1,
    shuffle = False,
))
print("done!")

res = []
for i, p in enumerate(scores3):
#     print(i, '/', len(test))
    res.append(p['class_ids'][0])   


done!


In [12]:
submission = pd.DataFrame(
    {"Id": test.index, "winPlacePerc": res},
    columns=["Id", "winPlacePerc"]
)
submission.to_csv("submission2.csv", index=False)